# Full Pipeline Comparison: Single-Channel vs Multi-Channel Dataset
## 1D-CNN Baseline — Stages 1, 2, 3

Trains the same **CNN_Classifier** pipeline on two 600-sample datasets under **equal sample conditions**:
- **Single-Channel**: channel 5 only — 100 samples per scenario
- **Multi-Channel**: 4 channels (c1, c3, c4, c7) — 25 samples per channel per scenario

| Channel | Center Freq | Bandwidth | Character |
|---|---|---|---|
| c1 | 3494.4 MHz | 499.2 MHz | Low-freq, narrowband |
| c3 | 4492.8 MHz | 499.2 MHz | Mid-freq, narrowband |
| c4 | 3993.6 MHz | 1331.2 MHz | Mid-freq, **wideband** |
| c7 | 6489.6 MHz | 1081.6 MHz | High-freq, **wideband** |

Full 3-stage pipeline per dataset:
```
Raw CIR → Stage 1 (CNN: LOS/NLOS?) → Stage 2 (RF: Single/Multi bounce?) → Stage 3 (RF: Predict bias) → d_corrected
```

Two levels of evaluation:
1. Single 70/15/15 split
2. Stratified 5-Fold CV (mean ± std)

In [ ]:
CONFIG = {
    'pre_crop': 10, 'post_crop': 50, 'total_len': 60,
    'search_start': 740, 'search_end': 890,
    'embedding_size': 128, 'input_channels': 1, 'dropout': 0.4,
    'batch_size': 64, 'max_epochs': 50, 'lr': 1e-3,
    'weight_decay': 1e-4, 'warmup_epochs': 3, 'patience': 40,
    'grad_clip': 1.0, 'val_ratio': 0.15, 'test_ratio': 0.15, 'seed': 42,
}

GROUND_TRUTH = {
    '7.79m':  {'d_direct': 7.79,  'd_bounce': 12.79, 'bias': 5.00},
    '10.77m': {'d_direct': 10.77, 'd_bounce': 16.09, 'bias': 5.32},
    '14m':    {'d_direct': 14.00, 'd_bounce': 16.80, 'bias': 2.80},
}
MEASURED_NLOS_BIAS = {'7.79m': 5.00, '10.77m': 5.32, '14m': 2.80}

PEAK_CONFIG = {
    'peak_prominence': 0.20, 'peak_min_distance': 5,
    'single_bounce_max_peaks': 2,
    'search_start': 740, 'search_end': 890,
}

DATA_DIR = '../dataset/channels/'

In [ ]:
import copy, math, contextlib, io, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from scipy.signal import find_peaks
from scipy.stats import kurtosis as scipy_kurtosis
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay,
    classification_report, roc_curve, auc,
    mean_absolute_error, mean_squared_error, r2_score
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    torch.cuda.empty_cache()
print(f'Device: {device}')
torch.manual_seed(CONFIG['seed'])
np.random.seed(CONFIG['seed'])

---
## Section 1: Data Loading, Preprocessing & Feature Extraction

In [ ]:
def get_distance_group(fname):
    match = re.match(r'^([\d.]+m)', str(fname))
    return match.group(1) if match else None


def get_roi_alignment(sig, search_start=CONFIG['search_start'],
                      search_end=CONFIG['search_end']):
    region = sig[search_start:search_end]
    if len(region) == 0:
        return np.argmax(sig)
    peak_idx = search_start + np.argmax(region)
    peak_val = sig[peak_idx]
    noise_section = sig[:search_start]
    if len(noise_section) > 10:
        threshold = max(np.mean(noise_section) + 3*np.std(noise_section), 0.05*peak_val)
    else:
        threshold = 0.05 * peak_val
    leading_edge = peak_idx
    for i in range(peak_idx, max(search_start - 20, 0), -1):
        if sig[i] < threshold:
            leading_edge = i + 1
            break
    return leading_edge


def _process_rows(df):
    """CIR preprocessing — outputs (N, 1, 60) channels-first for Conv1d."""
    PRE = CONFIG['pre_crop']; TOTAL = CONFIG['total_len']
    cir_cols = sorted([c for c in df.columns if c.startswith('CIR')],
                      key=lambda x: int(x.replace('CIR', '')))
    seqs, labels = [], []
    for _, row in df.iterrows():
        sig = pd.to_numeric(row[cir_cols], errors='coerce').fillna(0).astype(float).values
        rxpacc = float(row.get('RXPACC', 128.0))
        if rxpacc > 0:
            sig = sig / rxpacc
        le = get_roi_alignment(sig)
        start = max(0, le - PRE); end = start + TOTAL
        if end > len(sig): end = len(sig); start = max(0, end - TOTAL)
        crop = sig[start:end]
        if len(crop) < TOTAL:
            crop = np.pad(crop, (0, TOTAL - len(crop)), mode='constant')
        lo, hi = crop.min(), crop.max()
        crop = (crop - lo) / (hi - lo) if hi > lo else np.zeros(TOTAL)
        seqs.append(crop); labels.append(float(row['Label']))
    X = np.array(seqs).reshape(-1, 1, TOTAL).astype(np.float32)
    y = np.array(labels).astype(np.float32)
    return X, y


def extract_cir_features(sig, leading_edge):
    ss, se = PEAK_CONFIG['search_start'], PEAK_CONFIG['search_end']
    peak_idx = np.argmax(sig[ss:se]) + ss
    roi_start = max(0, leading_edge - 5)
    roi_end = min(len(sig), leading_edge + 120)
    roi = sig[roi_start:roi_end]
    if len(roi) > 0 and np.max(roi) > 0:
        roi_norm = roi / np.max(roi)
        peaks, _ = find_peaks(roi_norm, prominence=PEAK_CONFIG['peak_prominence'],
                              distance=PEAK_CONFIG['peak_min_distance'])
        num_peaks = len(peaks)
    else:
        num_peaks = 0
    return {'Num_Peaks': float(num_peaks)}


def extract_features_from_df(data_df):
    cir_cols = sorted([c for c in data_df.columns if c.startswith('CIR')],
                      key=lambda x: int(x.replace('CIR', '')))
    features_list, source_files, distances = [], [], []
    for _, row in data_df.iterrows():
        sig = pd.to_numeric(row[cir_cols], errors='coerce').fillna(0).astype(float).values
        rxpacc = float(row.get('RXPACC', 128.0))
        if rxpacc > 0:
            sig = sig / rxpacc
        le = get_roi_alignment(sig)
        feats = extract_cir_features(sig, le)
        features_list.append(feats)
        source_files.append(str(row['Source_File']))
        distances.append(float(row['Distance']))
    return pd.DataFrame(features_list), source_files, distances


print('Preprocessing & feature extraction functions ready.')

---
## Section 2: CNN_Classifier Architecture

In [ ]:
class CNN_Classifier(nn.Module):
    def __init__(self, input_channels=1, embedding_size=128, dropout=0.4):
        super().__init__()
        self.embedding_size = embedding_size
        self.encoder = nn.Sequential(
            nn.Conv1d(input_channels, 16, kernel_size=5, padding=2),
            nn.BatchNorm1d(16), nn.ReLU(),
            nn.Conv1d(16, 32, kernel_size=5, padding=2, stride=2),
            nn.BatchNorm1d(32), nn.ReLU(),
            nn.Conv1d(32, embedding_size, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm1d(embedding_size), nn.ReLU(),
        )
        self.gap = nn.AdaptiveAvgPool1d(1)
        self.classifier = nn.Sequential(
            nn.Linear(embedding_size, 32), nn.SiLU(),
            nn.Dropout(dropout), nn.Linear(32, 1), nn.Sigmoid()
        )

    def forward(self, x, return_embedding=False):
        features = self.encoder(x)
        embedding = self.gap(features).squeeze(-1)
        prediction = self.classifier(embedding)
        if return_embedding:
            return prediction, embedding
        return prediction

_m = CNN_Classifier()
print(f'CNN_Classifier | params: {sum(p.numel() for p in _m.parameters()):,} | embed_dim=128')
del _m

---
## Section 3: Pipeline Functions
Stage 1 training, evaluation, embedding extraction, and full pipeline runner.

In [ ]:
def train_s1(X_train, y_train, X_val, y_val, config=CONFIG, verbose=True, seed=None):
    _seed = seed if seed is not None else config['seed']
    torch.manual_seed(_seed); np.random.seed(_seed)
    X_tr = torch.tensor(X_train).to(device)
    y_tr = torch.tensor(y_train).unsqueeze(1).to(device)
    X_va = torch.tensor(X_val).to(device)
    y_va = torch.tensor(y_val).unsqueeze(1).to(device)
    loader = DataLoader(TensorDataset(X_tr, y_tr), batch_size=config['batch_size'], shuffle=True)
    model = CNN_Classifier(
        input_channels=config['input_channels'],
        embedding_size=config['embedding_size'],
        dropout=config['dropout']
    ).to(device)
    criterion = nn.BCELoss()
    optimizer = optim.AdamW(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])
    T = config['max_epochs']; W = config['warmup_epochs']
    def lr_lambda(e):
        if e < W: return (e+1)/W
        return max(0.01, 0.5*(1+math.cos(math.pi*(e-W)/max(1,T-W))))
    scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)
    history = {'train_loss':[], 'val_loss':[], 'train_acc':[], 'val_acc':[]}
    best_val_acc = 0; best_state = None; patience_counter = 0
    for epoch in range(T):
        model.train(); tl, tc, tt = 0, 0, 0
        for bx, by in loader:
            optimizer.zero_grad()
            pred = model(bx); loss = criterion(pred, by); loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), config['grad_clip'])
            optimizer.step()
            tl += loss.item()*len(bx); tc += ((pred>0.5).float()==by).sum().item(); tt += len(bx)
        model.eval()
        with torch.no_grad():
            vp = model(X_va); vl = criterion(vp, y_va).item()
            va = ((vp>0.5).float()==y_va).float().mean().item()
        history['train_loss'].append(tl/tt); history['val_loss'].append(vl)
        history['train_acc'].append(tc/tt); history['val_acc'].append(va)
        scheduler.step()
        if va > best_val_acc:
            best_val_acc = va; best_state = copy.deepcopy(model.state_dict()); patience_counter = 0
        else:
            patience_counter += 1
        if verbose and (epoch%10==0 or epoch==T-1):
            print(f'  Ep {epoch:>3} | Loss: {tl/tt:.4f} | Val Acc: {100*va:.2f}% | Best: {100*best_val_acc:.2f}%')
        if patience_counter >= config.get('patience', T):
            if verbose: print(f'  Early stopping at epoch {epoch}')
            break
    model.load_state_dict(best_state)
    return model, history


def evaluate_s1(model, X, y_true):
    model.eval()
    X_t = torch.tensor(X).to(device)
    with torch.no_grad():
        pred = model(X_t)
    y_prob = pred.cpu().numpy().flatten()
    y_pred = (y_prob > 0.5).astype(float)
    y_true = y_true.flatten()
    rep = classification_report(y_true, y_pred, target_names=['LOS','NLOS'],
                                output_dict=True, zero_division=0)
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    return {
        'acc': rep['accuracy'], 'f1_macro': rep['macro avg']['f1-score'],
        'f1_los': rep['LOS']['f1-score'], 'f1_nlos': rep['NLOS']['f1-score'],
        'auc': auc(fpr, tpr), 'fpr': fpr, 'tpr': tpr,
        'cm': confusion_matrix(y_true, y_pred),
    }


def extract_cnn_embeddings(model, data_df, batch_size=256):
    X, _ = _process_rows(data_df)
    X_tensor = torch.tensor(X).to(device)
    all_emb = []
    model.eval()
    with torch.no_grad():
        for i in range(0, len(X_tensor), batch_size):
            batch = X_tensor[i:i+batch_size]
            _, emb = model(batch, return_embedding=True)
            all_emb.append(emb.cpu().numpy())
    return np.vstack(all_emb)


print('Pipeline functions ready.')

In [ ]:
def run_pipeline(csv_path, label, config=CONFIG, verbose=True, seed=None):
    """Run full 3-stage pipeline. Returns dict with all metrics."""
    _seed = seed if seed is not None else config['seed']
    print(f"\n{'='*60}\nPipeline: {label}\n{'='*60}")

    # ---- Load & split DataFrame ----
    df = pd.read_csv(csv_path)
    if verbose: print(f'Loaded {len(df)} samples (LOS={int((df["Label"]==0).sum())}, NLOS={int((df["Label"]==1).sum())})')
    train_df, temp_df = train_test_split(df, test_size=config['val_ratio']+config['test_ratio'],
                                         stratify=df['Label'], random_state=_seed)
    val_df, test_df = train_test_split(temp_df, test_size=config['test_ratio']/(config['val_ratio']+config['test_ratio']),
                                       stratify=temp_df['Label'], random_state=_seed)
    train_df = train_df.reset_index(drop=True)
    val_df = val_df.reset_index(drop=True)
    test_df = test_df.reset_index(drop=True)
    if verbose: print(f'Split: Train={len(train_df)}, Val={len(val_df)}, Test={len(test_df)}')

    # ---- Stage 1: Train CNN ----
    if verbose: print(f'\nSTAGE 1: Training 1D-CNN...')
    X_tr, y_tr = _process_rows(train_df)
    X_va, y_va = _process_rows(val_df)
    X_te, y_te = _process_rows(test_df)
    model, history = train_s1(X_tr, y_tr, X_va, y_va, config=config, verbose=verbose, seed=_seed)
    s1 = evaluate_s1(model, X_te, y_te)
    s1['history'] = history
    if verbose: print(f"  Stage 1 Test: Acc={s1['acc']:.4f} | F1={s1['f1_macro']:.4f} | AUC={s1['auc']:.4f}")

    # Freeze
    model.eval()
    for p in model.parameters(): p.requires_grad = False

    # ---- Stage 2: RF Bounce Classifier (NLOS only) ----
    if verbose: print(f'\nSTAGE 2: Training RF Bounce Classifier...')
    nlos_train = train_df[train_df['Label'] == 1].reset_index(drop=True)
    nlos_val = val_df[val_df['Label'] == 1].reset_index(drop=True)
    nlos_test = test_df[test_df['Label'] == 1].reset_index(drop=True)

    feat_train_s2, _, _ = extract_features_from_df(nlos_train)
    feat_val_s2, _, _ = extract_features_from_df(nlos_val)
    feat_test_s2, src_test_nlos, dist_test_nlos = extract_features_from_df(nlos_test)

    threshold = PEAK_CONFIG['single_bounce_max_peaks']
    y_train_s2 = (feat_train_s2['Num_Peaks'] > threshold).astype(float).values
    y_val_s2 = (feat_val_s2['Num_Peaks'] > threshold).astype(float).values
    y_test_s2 = (feat_test_s2['Num_Peaks'] > threshold).astype(float).values

    X_train_emb = extract_cnn_embeddings(model, nlos_train)
    X_val_emb = extract_cnn_embeddings(model, nlos_val)
    X_test_emb = extract_cnn_embeddings(model, nlos_test)

    rf_s2 = RandomForestClassifier(n_estimators=200, max_depth=None,
        min_samples_split=5, min_samples_leaf=2, class_weight='balanced',
        random_state=_seed, n_jobs=-1)
    rf_s2.fit(X_train_emb, y_train_s2)
    s2_preds = rf_s2.predict(X_test_emb)
    s2_acc = (s2_preds == y_test_s2).mean()
    if verbose: print(f'  Stage 2 Test Accuracy: {100*s2_acc:.2f}% (NLOS samples: {len(nlos_test)})')

    # ---- Stage 3: RF Bias Regressor (single-bounce NLOS) ----
    if verbose: print(f'\nSTAGE 3: Training RF Bias Regressor...')
    def get_s3_idx(feat_df, source_files):
        indices, biases = [], []
        for i, fname in enumerate(source_files):
            grp = get_distance_group(fname)
            if grp not in MEASURED_NLOS_BIAS: continue
            if feat_df.iloc[i]['Num_Peaks'] > threshold: continue
            indices.append(i); biases.append(MEASURED_NLOS_BIAS[grp])
        return np.array(indices), np.array(biases, dtype=np.float32)

    feat_train_nlos, src_train_nlos, _ = extract_features_from_df(nlos_train)
    feat_val_nlos, src_val_nlos, _ = extract_features_from_df(nlos_val)
    train_s3_idx, y_train_s3 = get_s3_idx(feat_train_nlos, src_train_nlos)
    val_s3_idx, y_val_s3 = get_s3_idx(feat_val_nlos, src_val_nlos)
    test_s3_idx, y_test_s3 = get_s3_idx(feat_test_s2, src_test_nlos)

    X_train_s3 = X_train_emb[train_s3_idx]
    X_test_s3 = X_test_emb[test_s3_idx]

    rf_s3 = RandomForestRegressor(n_estimators=200, max_depth=None,
        min_samples_split=5, min_samples_leaf=2, random_state=_seed, n_jobs=-1)
    rf_s3.fit(X_train_s3, y_train_s3)

    s3_test_preds = rf_s3.predict(X_test_s3)
    s3_mae = mean_absolute_error(y_test_s3, s3_test_preds)
    if verbose: print(f'  Stage 3 Test MAE: {s3_mae:.4f}m (single-bounce samples: {len(X_test_s3)})')

    # ---- Build correction results ----
    results_rows = []
    for j, orig_i in enumerate(test_s3_idx):
        fname = src_test_nlos[orig_i]
        grp = get_distance_group(fname)
        gt = GROUND_TRUTH[grp]
        d_uwb = dist_test_nlos[orig_i]
        ml_bias = s3_test_preds[j]
        results_rows.append({
            'group': grp, 'd_uwb': d_uwb, 'd_direct': gt['d_direct'],
            'd_bounce': gt['d_bounce'], 'actual_bias': gt['bias'],
            'ml_bias': ml_bias, 'd_corrected': d_uwb - ml_bias,
            'bias_error': abs(ml_bias - gt['bias']),
            'correction_error': abs((d_uwb - ml_bias) - gt['d_direct']),
        })
    results_df = pd.DataFrame(results_rows) if results_rows else pd.DataFrame()
    correction_mae = results_df['correction_error'].mean() if len(results_df) > 0 else float('nan')

    if verbose:
        print(f'  Distance Correction MAE: {correction_mae:.4f}m')
        print(f"\n  Summary: S1 Acc={s1['acc']:.4f} | S2 Acc={s2_acc:.4f} | S3 MAE={s3_mae:.4f}m | Corr MAE={correction_mae:.4f}m")

    return {
        'label': label,
        's1': s1, 's2_acc': s2_acc, 's3_mae': s3_mae,
        'correction_mae': correction_mae, 'results_df': results_df,
        's2_cm': confusion_matrix(y_test_s2, s2_preds),
    }


print('run_pipeline() ready — full 3-stage CNN pipeline.')

---
## Section 4: Single 70/15/15 Split — Full Pipeline Comparison

In [ ]:
res_single = run_pipeline(DATA_DIR + 'single_channel5_dataset.csv',
                          'Single-Channel (c5 only)')
res_multi  = run_pipeline(DATA_DIR + 'multi_channel4_dataset.csv',
                          'Multi-Channel (c1,c3,c4,c7)')

In [ ]:
# ── Stage 1 Plots ──────────────────────────────────────────────────
_res = [res_single, res_multi]
_colors = ['#3498db', '#e74c3c']
width = 0.3

fig, axs = plt.subplots(1, 4, figsize=(24, 6))

# Bar: key metrics
ax = axs[0]
mk = ['acc','f1_macro','auc']; mn = ['Accuracy','Macro F1','AUC']
for i, (r, c) in enumerate(zip(_res, _colors)):
    vals = [r['s1'][k] for k in mk]
    bars = ax.bar(np.arange(3)+i*width, vals, width, label=r['label'], color=c, alpha=0.85)
    for bar, v in zip(bars, vals):
        ax.text(bar.get_x()+bar.get_width()/2, bar.get_height()+0.005,
                f'{v:.3f}', ha='center', va='bottom', fontsize=8, fontweight='bold')
ax.set_xticks(np.arange(3)+width/2); ax.set_xticklabels(mn)
ax.set_ylim([0.4,1.12]); ax.set_title('Stage 1 Metrics', fontweight='bold')
ax.legend(fontsize=8); ax.grid(True, alpha=0.3, axis='y')

# ROC
ax = axs[1]
for r, c in zip(_res, _colors):
    ax.plot(r['s1']['fpr'], r['s1']['tpr'], color=c, lw=2,
            label=f"{r['label']} (AUC={r['s1']['auc']:.3f})")
ax.plot([0,1],[0,1],'k--',lw=1,alpha=0.5)
ax.set_title('Stage 1 ROC', fontweight='bold'); ax.set_xlabel('FPR'); ax.set_ylabel('TPR')
ax.legend(fontsize=8); ax.grid(True, alpha=0.3)

# Val acc curves
ax = axs[2]
for r, c in zip(_res, _colors):
    ax.plot(r['s1']['history']['val_acc'], color=c, lw=2, label=r['label'])
ax.set_title('Stage 1 Val Accuracy', fontweight='bold')
ax.set_xlabel('Epoch'); ax.set_ylabel('Val Accuracy')
ax.set_ylim([0.4,1.05]); ax.legend(fontsize=8); ax.grid(True, alpha=0.3)

# Per-class F1
ax = axs[3]
for i, (r, c) in enumerate(zip(_res, _colors)):
    vals = [r['s1']['f1_los'], r['s1']['f1_nlos']]
    bars = ax.bar(np.arange(2)+i*width, vals, width, label=r['label'], color=c, alpha=0.85)
    for bar, v in zip(bars, vals):
        ax.text(bar.get_x()+bar.get_width()/2, bar.get_height()+0.005,
                f'{v:.3f}', ha='center', va='bottom', fontsize=8, fontweight='bold')
ax.set_xticks(np.arange(2)+width/2); ax.set_xticklabels(['LOS F1','NLOS F1'])
ax.set_ylim([0.4,1.12]); ax.set_title('Per-Class F1', fontweight='bold')
ax.legend(fontsize=8); ax.grid(True, alpha=0.3, axis='y')

plt.suptitle('1D-CNN — Stage 1: Single-Channel vs Multi-Channel — Single Split',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout(); plt.show()

In [ ]:
# ── Stages 2 & 3 Plots ────────────────────────────────────────────
fig, axs = plt.subplots(1, 3, figsize=(20, 6))

# Stage 2 accuracy bar
ax = axs[0]
for i, (r, c) in enumerate(zip(_res, _colors)):
    bar = ax.bar(i, r['s2_acc'], 0.6, label=r['label'], color=c, alpha=0.85)
    ax.text(i, r['s2_acc']+0.01, f"{100*r['s2_acc']:.1f}%",
            ha='center', fontsize=12, fontweight='bold')
ax.set_xticks(range(len(_res))); ax.set_xticklabels([r['label'] for r in _res], fontsize=9)
ax.set_ylim([0.4,1.12]); ax.set_ylabel('Accuracy')
ax.set_title('Stage 2: Bounce Classification', fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

# Stage 3 MAE bar
ax = axs[1]
for i, (r, c) in enumerate(zip(_res, _colors)):
    bar = ax.bar(i, r['s3_mae'], 0.6, label=r['label'], color=c, alpha=0.85)
    ax.text(i, r['s3_mae']+0.002, f"{r['s3_mae']:.4f}m",
            ha='center', fontsize=12, fontweight='bold')
ax.set_xticks(range(len(_res))); ax.set_xticklabels([r['label'] for r in _res], fontsize=9)
ax.set_ylabel('MAE (m)')
ax.set_title('Stage 3: Bias Prediction MAE', fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

# Correction MAE bar
ax = axs[2]
for i, (r, c) in enumerate(zip(_res, _colors)):
    bar = ax.bar(i, r['correction_mae'], 0.6, label=r['label'], color=c, alpha=0.85)
    ax.text(i, r['correction_mae']+0.05, f"{r['correction_mae']:.3f}m",
            ha='center', fontsize=12, fontweight='bold')
ax.set_xticks(range(len(_res))); ax.set_xticklabels([r['label'] for r in _res], fontsize=9)
ax.set_ylabel('MAE (m)')
ax.set_title('Distance Correction MAE', fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

plt.suptitle('1D-CNN — Stages 2 & 3: Single-Channel vs Multi-Channel — Single Split',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout(); plt.show()

# Summary table
print('\n' + '='*65)
print(f"{'Metric':<25} {'Single-Ch':>12} {'Multi-Ch':>12}  Delta")
print('='*65)
for k, n in [('acc','S1 Accuracy'),('f1_macro','S1 Macro F1'),('auc','S1 AUC')]:
    sv=res_single['s1'][k]; mv=res_multi['s1'][k]; d=mv-sv
    print(f"{n:<25} {sv:>12.4f} {mv:>12.4f}   {'\u25b2' if d>0 else '\u25bc'}{abs(d):.4f}")
for k, n, flip in [('s2_acc','S2 Bounce Acc',False),('s3_mae','S3 Bias MAE',True),('correction_mae','Correction MAE',True)]:
    sv=res_single[k]; mv=res_multi[k]; d=mv-sv
    better = d<0 if flip else d>0
    print(f"{n:<25} {sv:>12.4f} {mv:>12.4f}   {'\u25b2' if better else '\u25bc'}{abs(d):.4f}")
print('='*65)

---
## Section 5: Stratified 5-Fold Cross-Validation — Full Pipeline
More reliable estimate: averages over 5 different train/test splits across all 3 stages.

In [ ]:
def run_kfold_pipeline(csv_path, label, n_splits=5, config=CONFIG, seed=42):
    print(f"\n{'='*60}\n5-Fold CV Pipeline: {label}\n{'='*60}")
    df = pd.read_csv(csv_path)
    print(f'Loaded {len(df)} samples')
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    fold_metrics = []

    for fold, (tv_idx, te_idx) in enumerate(skf.split(df, df['Label'])):
        df_tv = df.iloc[tv_idx].reset_index(drop=True)
        df_te = df.iloc[te_idx].reset_index(drop=True)
        df_tr, df_va = train_test_split(df_tv, test_size=0.15, stratify=df_tv['Label'], random_state=seed)
        df_tr = df_tr.reset_index(drop=True); df_va = df_va.reset_index(drop=True)

        fold_seed = seed + fold
        # Stage 1
        X_tr, y_tr = _process_rows(df_tr)
        X_va, y_va = _process_rows(df_va)
        X_te, y_te = _process_rows(df_te)
        with contextlib.redirect_stdout(io.StringIO()):
            model, _ = train_s1(X_tr, y_tr, X_va, y_va, config=config, verbose=False, seed=fold_seed)
        s1 = evaluate_s1(model, X_te, y_te)
        model.eval()
        for p in model.parameters(): p.requires_grad = False

        # Stage 2
        nlos_tr = df_tr[df_tr['Label']==1].reset_index(drop=True)
        nlos_te = df_te[df_te['Label']==1].reset_index(drop=True)
        feat_tr_s2, _, _ = extract_features_from_df(nlos_tr)
        feat_te_s2, src_te, dist_te = extract_features_from_df(nlos_te)
        thr = PEAK_CONFIG['single_bounce_max_peaks']
        y_tr_s2 = (feat_tr_s2['Num_Peaks'] > thr).astype(float).values
        y_te_s2 = (feat_te_s2['Num_Peaks'] > thr).astype(float).values
        emb_tr = extract_cnn_embeddings(model, nlos_tr)
        emb_te = extract_cnn_embeddings(model, nlos_te)
        rf2 = RandomForestClassifier(n_estimators=200, max_depth=None,
            min_samples_split=5, min_samples_leaf=2, class_weight='balanced',
            random_state=fold_seed, n_jobs=-1)
        rf2.fit(emb_tr, y_tr_s2)
        s2_acc = (rf2.predict(emb_te) == y_te_s2).mean()

        # Stage 3
        def _s3_idx(feat_df, src_files):
            idx, bias = [], []
            for i, fn in enumerate(src_files):
                g = get_distance_group(fn)
                if g not in MEASURED_NLOS_BIAS: continue
                if feat_df.iloc[i]['Num_Peaks'] > thr: continue
                idx.append(i); bias.append(MEASURED_NLOS_BIAS[g])
            return np.array(idx), np.array(bias, dtype=np.float32)

        feat_tr_nlos, src_tr, _ = extract_features_from_df(nlos_tr)
        tr3_idx, y_tr3 = _s3_idx(feat_tr_nlos, src_tr)
        te3_idx, y_te3 = _s3_idx(feat_te_s2, src_te)

        s3_mae = float('nan')
        correction_mae = float('nan')
        if len(tr3_idx) > 0 and len(te3_idx) > 0:
            rf3 = RandomForestRegressor(n_estimators=200, max_depth=None,
                min_samples_split=5, min_samples_leaf=2, random_state=fold_seed, n_jobs=-1)
            rf3.fit(emb_tr[tr3_idx], y_tr3)
            s3_preds = rf3.predict(emb_te[te3_idx])
            s3_mae = mean_absolute_error(y_te3, s3_preds)
            # Correction
            corr_errors = []
            for j, oi in enumerate(te3_idx):
                g = get_distance_group(src_te[oi])
                gt = GROUND_TRUTH[g]
                d_corr = dist_te[oi] - s3_preds[j]
                corr_errors.append(abs(d_corr - gt['d_direct']))
            correction_mae = np.mean(corr_errors)

        fm = {'s1_acc': s1['acc'], 's1_f1': s1['f1_macro'], 's1_auc': s1['auc'],
              's2_acc': s2_acc, 's3_mae': s3_mae, 'correction_mae': correction_mae}
        fold_metrics.append(fm)
        print(f"  Fold {fold+1}/{n_splits} | S1={s1['acc']:.3f} | S2={s2_acc:.3f} | S3={s3_mae:.4f}m | Corr={correction_mae:.3f}m")

    summary = {'label': label}
    print(f"\n  {'\u2500'*55}")
    print(f"  {'Metric':<18} {'Mean':>8} {'Std':>8}")
    print(f"  {'\u2500'*55}")
    for key in ['s1_acc','s1_f1','s1_auc','s2_acc','s3_mae','correction_mae']:
        vals = np.array([m[key] for m in fold_metrics])
        valid = vals[~np.isnan(vals)]
        summary[key] = {'mean': valid.mean(), 'std': valid.std(), 'all': vals.tolist()}
        print(f"  {key:<18} {valid.mean():>8.4f} {valid.std():>8.4f}")
    print(f"  {'\u2500'*55}")
    return summary


kfold_single = run_kfold_pipeline(DATA_DIR + 'single_channel5_dataset.csv', 'Single-Channel (c5 only)')
kfold_multi  = run_kfold_pipeline(DATA_DIR + 'multi_channel4_dataset.csv',  'Multi-Channel (c1,c3,c4,c7)')

In [ ]:
# ── K-Fold comparison plots ────────────────────────────────────────
_kf = [kfold_single, kfold_multi]
_colors = ['#3498db', '#e74c3c']
_mk = ['s1_acc','s1_f1','s1_auc','s2_acc','s3_mae','correction_mae']
_mn = ['S1 Accuracy','S1 Macro F1','S1 AUC','S2 Bounce Acc','S3 Bias MAE','Correction MAE']
width = 0.3

fig, axs = plt.subplots(1, 2, figsize=(20, 7))

# Bar with error bars
ax = axs[0]
x = np.arange(len(_mk))
for i, (kf, c) in enumerate(zip(_kf, _colors)):
    means = [kf[m]['mean'] for m in _mk]
    stds  = [kf[m]['std']  for m in _mk]
    bars = ax.bar(x+i*width, means, width, yerr=stds, label=kf['label'],
                  color=c, alpha=0.85, capsize=4, error_kw={'elinewidth':1.5})
    for bar, m, s in zip(bars, means, stds):
        ax.text(bar.get_x()+bar.get_width()/2, bar.get_height()+s+0.008,
                f'{m:.3f}', ha='center', va='bottom', fontsize=7, fontweight='bold')
ax.set_xticks(x+width/2); ax.set_xticklabels(_mn, rotation=15, ha='right')
ax.set_title('5-Fold CV: Mean \u00b1 Std — Full Pipeline', fontweight='bold')
ax.set_ylabel('Score / MAE'); ax.legend(fontsize=9); ax.grid(True, alpha=0.3, axis='y')

# Box plots
ax = axs[1]
all_data, all_colors, positions = [], [], []
for mi, metric in enumerate(_mk):
    for ki, (kf, c) in enumerate(zip(_kf, _colors)):
        positions.append(mi*3 + ki)
        vals = [v for v in kf[metric]['all'] if not np.isnan(v)]
        all_data.append(vals)
        all_colors.append(c)
bp = ax.boxplot(all_data, positions=positions, widths=0.7, patch_artist=True,
                medianprops={'color':'black','linewidth':2})
for patch, c in zip(bp['boxes'], all_colors):
    patch.set_facecolor(c); patch.set_alpha(0.7)
ax.set_xticks([mi*3+0.5 for mi in range(len(_mk))])
ax.set_xticklabels(_mn, rotation=15, ha='right')
ax.set_title('Score Distribution per Fold (blue=single, red=multi)', fontweight='bold')
ax.set_ylabel('Score / MAE'); ax.grid(True, alpha=0.3, axis='y')

plt.suptitle('1D-CNN — Full Pipeline: Stratified 5-Fold CV — Single vs Multi Channel',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout(); plt.show()

# Summary table
print('\n' + '='*70)
print(f"{'Metric':<18} {'Single Mean\u00b1Std':>20} {'Multi Mean\u00b1Std':>20}  Delta")
print('='*70)
for k, n in zip(_mk, _mn):
    sm, ss = kfold_single[k]['mean'], kfold_single[k]['std']
    mm, ms = kfold_multi[k]['mean'],  kfold_multi[k]['std']
    d = mm - sm
    print(f"{n:<18} {sm:>8.4f}\u00b1{ss:.4f}      {mm:>8.4f}\u00b1{ms:.4f}   {'\u25b2' if d>0 else '\u25bc'}{abs(d):.4f}")
print('='*70)